In [ ]:
library(tidyverse)
library(targets)
library(tarchetypes)

In [ ]:
deg_recipe = qs::qread('deg_restored_recipe.qs')

In [ ]:
deg_recipe

In [ ]:
df_t = tibble(output_name = c("combined_deg_seurat", "combined_gsea_seurat", "combined_gost_seurat"),
              name_prefix = c("deg_ensmus_", "gsea_", 'gost_result_'))

combined_recipe_degs = deg_recipe %>%
    select(deg_output_suffix) %>%
    tidyr::crossing(df_t) %>%
    mutate(results_name = paste0(name_prefix, deg_output_suffix)) %>%
    group_by(output_name) %>%
    mutate(results_name = list(unique(results_name))) %>%
    distinct(results_name, .keep_all = TRUE)


In [ ]:
qs::qsave(combined_recipe_degs, "combined_recipe_degs.qs")
combined_recipe_degs

In [ ]:
combined_recipe_degs %>% pull(output_name)

In [ ]:
combined_recipe_degs %>% filter(output_name == 'combined_deg_seurat') %>% pull(results_name) %>% unlist %>% tibble(results_name=.) %>% filter(results_name == 'deg_ensmus_Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none')

In [ ]:
combined_recipe_nhgc = deg_recipe %>%
    select(nhgc_obj, comparison) %>%
    mutate(results_name = paste0(nhgc_obj)) %>%
    group_by(comparison) %>%
    mutate(results_name = list(unique(results_name))) %>%
    distinct(results_name, .keep_all = TRUE) %>%
    mutate(output_name = paste0('combined_nhgc_', comparison)) %>% 
    ungroup %>%
    select(output_name, results_name)


In [ ]:
qs::qsave(combined_recipe_nhgc, "combined_recipe_nhgc.qs")
combined_recipe_nhgc

In [ ]:
combined_recipe_nhgc %>% pull(output_name) %>% dput

In [ ]:
qs::qread('_targets/objects/combined_deg_seurat_formatted')

In [ ]:
qs::qread('_targets_MAST/objects/combined_deg_seurat_formatted') %>%
readr::write_excel_csv('combined_deg_seurat_formatted.csv')

In [ ]:
qs::qread('_targets_MAST/objects/combined_gsea_seurat_formatted') %>%
readr::write_excel_csv('combined_gsea_seurat_formatted.csv')

In [ ]:
qs::qread('_targets_MAST/objects/combined_gost_seurat_formatted') %>%
filter(source %in% c("GO:BP", "GO:MF", "GO:CC", "REAC"))  %>%
select(-precision, -recall, -effective_domain_size, -source_order, -parents, -evidence_codes, -intersection) %>% 
filter(cells_b %in% c("neg", "neg_restored")) %>%
readr::write_excel_csv('combined_gost_seurat_formatted.csv')

In [ ]:
targets::tar_meta() %>%
filter(name == 'combined_gsea_seurat')